## Check List
- Chèn text tĩnh
- Chèn text động
- Chèn text theo thời gian
- Chèn text nhiều dòng
### filter
- drawtext
### Other
- Nhúng phụ đề


In [1]:
import dotenv

dotenv.load_dotenv(dotenv.find_dotenv())

True

In [2]:
import os
FFMPEG_PATH = os.getenv("FFMPEG_PATH")
FFMPEG_PATH

'C:/Users/ADMIN/AppData/Local/Microsoft/WinGet/Packages/Gyan.FFmpeg_Microsoft.Winget.Source_8wekyb3d8bbwe/ffmpeg-7.1.1-full_build/bin/ffmpeg.exe'

In [3]:
import subprocess
import logging

logger = logging.getLogger(__name__)


def runf(args: list):
    args[0] = FFMPEG_PATH
    logger.info("Running FFmpeg: %s", " ".join(args))
    result = subprocess.run(args, capture_output=True, text=True)
    
    if result.returncode != 0:
        logger.error("FFmpeg failed:\n%s", result.stderr)
        raise RuntimeError("FFmpeg command failed.")
    
    logger.debug("FFmpeg output:\n%s", result.stdout)
    return result.stdout

In [4]:
def add_text(input_path, output_path, text,
             font_path, font_size=48, font_color="white",
             x="(w-text_w)/2", y="(h-text_h)/2",
             start_time=None, end_time=None,
             animation=None):
    """
    Thêm text tĩnh hoặc động vào video.
    animation:
        None -> text tĩnh
        str  -> dùng prebuilt animation
        dict -> custom animation params (ghi đè x, y, fontcolor, fontsize, enable)
    """
    if not os.path.exists(font_path):
        raise FileNotFoundError(f"Font file not found: {font_path}")

    # Escape text và font path
    safe_text = text.replace(" ", r"\ ").replace("'", r"\'")
    safe_font_path = font_path.replace("\\", "/").replace(":", r"\:")

    # Prebuilt animation library
    prebuilt = {
        "move_right": {"x": "10*t"},
        "move_up": {"y": "h-(t*50)"},
        "fade_in": {"fontcolor": f"{font_color}@'min(1,(t/2))'"},
        "blink": {"enable": "mod(t,1) < 0.5"},
        "circle": {
            "x": "w/2+100*cos(t*2)-text_w/2",
            "y": "h/2+100*sin(t*2)-text_h/2"
        },
        "zoom_in": {"fontsize": "min(100, 20 + t*10)"},
        "shake": {"x": "(w-text_w)/2 + 10*sin(t*20)"}
    }

    # Animation params
    if isinstance(animation, str) and animation in prebuilt:
        anim_params = prebuilt[animation]
    elif isinstance(animation, dict):
        anim_params = animation
    else:
        anim_params = {}

    # Apply animation overrides
    x = anim_params.get("x", x)
    y = anim_params.get("y", y)
    font_color = anim_params.get("fontcolor", font_color)
    font_size = anim_params.get("fontsize", font_size)
    enable = anim_params.get("enable", None)

    # Time-based enable
    if start_time is not None and end_time is not None:
        time_enable = f"between(t,{start_time},{end_time})"
        enable = time_enable if not enable else f"({enable})*({time_enable})"

    # Build filter string
    drawtext_filter = (
        f"drawtext=fontfile='{safe_font_path}':"
        f"text={safe_text}:"
        f"fontcolor={font_color}:"
        f"fontsize={font_size}:"
        f"x={x}:y={y}"
    )
    if enable:
        drawtext_filter += f":enable='{enable}'"

    logger.debug(f"DrawText filter: {drawtext_filter}")

    args = [
        "ffmpeg",
        "-i", input_path,
        "-vf", drawtext_filter,
        "-c:a", "copy",
        output_path
    ]
    return runf(args)


In [ ]:
input_path="D:/ThienPV/code/demo/.trash/TestAPI.mp4"
output_path="D:/ThienPV/code/demo/.trash/TestAPI_text.mp4"
text= "Hello World"
font_path="D:/ThienPV/code/demo/assets/font/Montserrat/static/Montserrat-Medium.ttf"
start_time=1
end_time=5

add_text(input_path=input_path, 
         output_path=output_path, 
         text=text,
        font_path=font_path, 
        font_size=48, 
        font_color="white",
        x="(w-text_w)/2",
        y="(h-text_h)/2",
        start_time=0, 
        end_time=8,
        animation="movie_right")

# Moviepy

## Import

In [35]:
from __future__ import annotations
from dataclasses import dataclass, field
from typing import Dict, Any, Tuple, Optional, Union
from moviepy import TextClip, ColorClip, CompositeVideoClip, VideoClip
from math import ceil

## dataclass

In [62]:
Rect = Tuple[int, int, int, int]  # (x, y, w, h)
Payload = Union[str, VideoClip]

@dataclass
class Fonts:
    sans: Optional[str] = None           # path to .ttf/.otf; None => Pillow default
    mono: Optional[str] = None

@dataclass
class Canvas:
    size: Tuple[int, int] = (1920, 1080)
    title_safe_inset: int = 96
    action_safe_inset: int = 48

@dataclass
class Style:
    color: str = "white"
    stroke_color: Optional[str] = None
    stroke_width: int = 0
    opacity: float = 1.0
    pad: int = 0  # used for captions bg
    interline: int = 4  # line spacing between wrapped lines
    baseline_pad_pct: float = 0.32  # extra bottom padding to avoid descender/diacritic clipping
    top_pad_pct: float = 0.10  # small top padding for symmetry
@dataclass
class RenderConfig:
    fonts: Fonts = field(default_factory=Fonts)
    canvas: Canvas = field(default_factory=Canvas)

## core

In [61]:
from fontTools.ttLib import TTFont

def suggest_baseline_pad_pct(font_path: str, fudge: float = 0.06) -> float:
    tt = TTFont(font_path)
    upm = tt["head"].unitsPerEm
    hhea = tt["hhea"]
    desc = abs(hhea.descent)
    return (desc / upm) + fudge  # ví dụ Inter ~0.24 + 0.06 = 0.30


In [82]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np
from moviepy import ImageClip

def oversized_text_clip(text, font_path, font_size, color="white",
                         bottom_extra_pct=0.4, width=None, align="left"):
    font = ImageFont.truetype(font_path, font_size)
    ascent, descent = font.getmetrics()
    w, h = font.getmask(text).size  # h = ascent + descent
    # ép canvas lớn hơn
    extra_bottom = int(font_size * bottom_extra_pct)
    canvas_h = h + extra_bottom
    if width is None:
        width = w
    img = Image.new("RGBA", (width, canvas_h), (0, 0, 0, 0))
    draw = ImageDraw.Draw(img)
    # dịch xuống vài pixel để chừa đáy
    draw.text((0, 0), text, font=font, fill=color)
    arr = np.array(img)
    return ImageClip(arr)



In [83]:
def make_text(text, font_path, px, safe=True):
    if safe:
        return oversized_text_clip(text, font_path, px, bottom_extra_pct=0.40)
    else:
        return TextClip(text=text, font=font_path, font_size=px, method="label", color="white", transparent=True)

In [87]:
# --- replace this in your module ---
from math import ceil
from moviepy import TextClip, ColorClip, CompositeVideoClip

def _pad_transparent(clip, left, top, right, bottom, bg_opacity=0.01):
    """Pad bằng nền gần như trong suốt để mask mở rộng thật (tránh crop alpha)."""
    W, H = clip.w + left + right, clip.h + top + bottom
    bg = ColorClip(size=(W, H), color=(0,0,0)).with_opacity(bg_opacity).with_duration(clip.duration or None)
    return CompositeVideoClip([bg, clip.with_position((left, top))], size=(W, H))

def _mk_text_clip(
    text: str,
    rect: tuple[int,int,int,int],
    font_path: str | None,
    px: int,
    text_align: str,
    style
) -> TextClip:
    """
    Safe text renderer:
    - 'wrap' (caption) khi có xuống dòng hoặc quá dài so với width rect
    - 'line' (label) cho dòng đơn để tránh crop descender
    - luôn pad top/bottom theo style để chống cắt
    """
    x, y, w, h = rect
    # Heuristic chọn mode, có thể override bằng style.mode_hint = "wrap" | "line"
    mode_hint = getattr(style, "mode_hint", None)
    is_multiline_input = ("\n" in text)
    heuristic_wrap = is_multiline_input or (w and len(text) > 22)  # tùy chỉnh theo ngữ cảnh
    use_wrap = (mode_hint == "wrap") or (mode_hint is None and heuristic_wrap)

    # Render
    if use_wrap:
        # caption: bọc theo width, dùng interline, tôn trọng text_align
        base = TextClip(
            text=text,
            font=font_path,
            font_size=px,
            color=style.color,
            method="caption",
            size=(w, None),
            text_align=text_align,
            interline=getattr(style, "interline", 6),
            stroke_color=style.stroke_color,
            stroke_width=style.stroke_width,
            transparent=True
        )
    else:
        # label: dòng đơn, bbox “sạch” hơn cho descender
        base = TextClip(
            text=text,
            font=font_path,
            font_size=px,
            color=style.color,
            method="label",
            # size không có ý nghĩa với label
            stroke_color=style.stroke_color,
            stroke_width=style.stroke_width,
            transparent=True
        )

    # Pad an toàn (top/bottom) để tránh cắt descender/diacritics
    top_pct  = max(0.06, getattr(style, "top_pad_pct", 0.10))
    bot_pct  = max(0.24, getattr(style, "baseline_pad_pct", 0.32))
    pad_top    = int(max(2, ceil(px * top_pct)))
    pad_bottom = int(max(4, ceil(px * bot_pct)))

    # Dùng mask-safe pad (alpha rất nhỏ) để mask mở rộng thật sự
    bg_opacity = 0.01  # gần như vô hình
    clip = _pad_transparent(base, 0, pad_top, 0, pad_bottom, bg_opacity=bg_opacity)

    return clip


def _place_in_rect(clip: VideoClip, rect: Rect, h_align: str, v_align: str) -> VideoClip:
    x, y, w, h = rect
    if h_align == "center":
        px = x + max(0, (w - clip.w)//2)
    elif h_align == "right":
        px = x + max(0, (w - clip.w))
    else:
        px = x
    if v_align == "center":
        py = y + max(0, (h - clip.h)//2)
    elif v_align == "bottom":
        py = y + max(0, (h - clip.h))
    else:
        py = y
    return clip.with_position((px, py))

def _fit_into_rect(clip: VideoClip, rect: Rect) -> VideoClip:
    _, _, w, h = rect
    if clip.w > w or clip.h > h:
        scale = max(clip.w / w, clip.h / h)
        clip = clip.resized(new_size=(int(clip.w/scale), int(clip.h/scale)))
    return clip

def _caption_bg(text_clip: VideoClip, pad: int, opacity: float = 0.65) -> VideoClip:
    W, H = text_clip.w + pad*2, text_clip.h + pad*2
    bg = ColorClip(size=(W, H), color=(0,0,0)).with_opacity(opacity).with_duration(text_clip.duration or None)
    return CompositeVideoClip([bg, text_clip.with_position((pad,pad))], size=(W, H))

# def _pad_transparent(clip: VideoClip, left: int, top: int, right: int, bottom: int) -> VideoClip:
#     W, H = clip.w + left + right, clip.h + top + bottom
#     bg = ColorClip(size=(W, H), color=(0,0,0)).with_opacity(0.0).with_duration(clip.duration or None)
#     return CompositeVideoClip([bg, clip.with_position((left, top))], size=(W, H))

def render_text_element(element_id: str,
                        payload: Payload,
                        spec: Dict[str, Any],
                        variant: str = "primary",
                        fonts: Optional[Fonts] = None,
                        style: Optional[Style] = None,
                        duration: Optional[float] = None,
                        debug_rect: bool = False) -> VideoClip:
    fonts = fonts or Fonts()
    style = style or Style()

    t = next(x for x in spec["types"] if x["id"] == element_id)
    layout = t["layout"]["alternate" if variant == "alternate" else "primary"]
    rect: Rect = tuple(layout["rect"])
    h_align: str = layout.get("align", "left")
    px = int(t["size"]["common"])

    if isinstance(payload, str):
        is_mono = bool(t["size"].get("mono"))
        font_path = (fonts.mono if is_mono else fonts.sans)
        base: VideoClip = _mk_text_clip(payload, rect, font_path, px, h_align, style)
        pad_top = int(max(2, ceil(px * style.top_pad_pct)))
        pad_bottom = int(max(4, ceil(px * style.baseline_pad_pct)))
        clip: VideoClip = _pad_transparent(base, 0, pad_top, 0, pad_bottom)
    else:
        clip = payload
        clip = _fit_into_rect(clip, rect)

    if element_id in {"section_marker", "equation", "quiz_question", "quiz_feedback", "cta"}:
        v_align = "center"
    elif element_id in {"captions"}:
        v_align = "bottom"
    else:
        v_align = "top"

    if element_id == "captions" and isinstance(payload, str):
        clip = _caption_bg(clip, pad=max(style.pad, 14), opacity=0.65)

    if style.opacity != 1.0:
        clip = clip.with_opacity(style.opacity)
    if duration is not None:
        clip = clip.with_duration(duration)

    clip = _place_in_rect(clip, rect, h_align=h_align, v_align=v_align)

    if debug_rect:
        x, y, w, h = rect
        box = ColorClip((w, h), color=(255, 255, 255)).with_opacity(0.15).with_duration(clip.duration or duration)
        box = box.with_position((x, y))
        return CompositeVideoClip([box, clip], size=(1920, 1080))

    return clip

## Wrapper

In [88]:
def title_clip(payload: Payload, spec: Dict[str, Any], **kw) -> VideoClip: return render_text_element("course_title", payload, spec, **kw)
def subtitle_clip(payload: Payload, spec: Dict[str, Any], **kw) -> VideoClip: return render_text_element("subtitle", payload, spec, **kw)
def lowerthird_clip(payload: Payload, spec: Dict[str, Any], **kw) -> VideoClip: return render_text_element("lowerthird_name", payload, spec, **kw)
def section_marker_clip(payload: Payload, spec: Dict[str, Any], **kw) -> VideoClip: return render_text_element("section_marker", payload, spec, **kw)
def heading_h1_clip(payload: Payload, spec: Dict[str, Any], **kw) -> VideoClip: return render_text_element("heading_h1", payload, spec, **kw)
def heading_h2_clip(payload: Payload, spec: Dict[str, Any], **kw) -> VideoClip: return render_text_element("heading_h2", payload, spec, **kw)
def heading_h3_clip(payload: Payload, spec: Dict[str, Any], **kw) -> VideoClip: return render_text_element("heading_h3", payload, spec, **kw)
def body_bullet_clip(payload: Payload, spec: Dict[str, Any], **kw) -> VideoClip: return render_text_element("body_bullet", payload, spec, **kw)
def definition_term_clip(payload: Payload, spec: Dict[str, Any], **kw) -> VideoClip: return render_text_element("definition_term", payload, spec, **kw)
def step_label_clip(payload: Payload, spec: Dict[str, Any], **kw) -> VideoClip: return render_text_element("step_label", payload, spec, **kw)
def code_snippet_clip(payload: Payload, spec: Dict[str, Any], **kw) -> VideoClip: return render_text_element("code_snippet", payload, spec, **kw)
def equation_clip(payload: Payload, spec: Dict[str, Any], **kw) -> VideoClip: return render_text_element("equation", payload, spec, **kw)
def chart_labels_clip(payload: Payload, spec: Dict[str, Any], **kw) -> VideoClip: return render_text_element("chart_labels", payload, spec, **kw)
def data_labels_clip(payload: Payload, spec: Dict[str, Any], **kw) -> VideoClip: return render_text_element("data_labels", payload, spec, **kw)
def captions_clip(payload: Payload, spec: Dict[str, Any], **kw) -> VideoClip: return render_text_element("captions", payload, spec, **kw)
def speaker_label_clip(payload: Payload, spec: Dict[str, Any], **kw) -> VideoClip: return render_text_element("speaker_label", payload, spec, **kw)
def quiz_question_clip(payload: Payload, spec: Dict[str, Any], **kw) -> VideoClip: return render_text_element("quiz_question", payload, spec, **kw)
def quiz_choices_clip(payload: Payload, spec: Dict[str, Any], **kw) -> VideoClip: return render_text_element("quiz_choices", payload, spec, **kw)
def quiz_feedback_clip(payload: Payload, spec: Dict[str, Any], **kw) -> VideoClip: return render_text_element("quiz_feedback", payload, spec, **kw)
def cta_clip(payload: Payload, spec: Dict[str, Any], **kw) -> VideoClip: return render_text_element("cta", payload, spec, **kw)
def progress_ui_clip(payload: Payload, spec: Dict[str, Any], **kw) -> VideoClip: return render_text_element("progress_ui", payload, spec, **kw)
def timestamp_clip(payload: Payload, spec: Dict[str, Any], **kw) -> VideoClip: return render_text_element("timestamp", payload, spec, **kw)
def legal_credits_clip(payload: Payload, spec: Dict[str, Any], **kw) -> VideoClip: return render_text_element("legal_credits", payload, spec, **kw)

## Test

In [89]:
from moviepy import CompositeVideoClip, ColorClip
import json
from pathlib import Path
from moviepy import concatenate_videoclips
import numpy as np

SPEC_PATH = Path("D:/ThienPV/code/demo/assets/elearning-text-type.json")  # place the JSON next to this script
if not SPEC_PATH.exists():
    # fallback to sandbox path if running in provided environment
    alt = Path("/mnt/data/e_learning_text_types_v2.json")
    SPEC_PATH = alt if alt.exists() else SPEC_PATH

with SPEC_PATH.open("r", encoding="utf-8") as f:
    SPEC = json.load(f)

# Set your font files here (recommended). If not set, Pillow default is used.
FONTS = Fonts(
    sans="D:/ThienPV/code/demo/assets/Inter/Inter-Italic-VariableFont_opsz,wght.ttf",
    mono="D:/ThienPV/code/demo/assets/font/JetBrains_Mono/static/JetBrainsMono-Regular.ttf"
)
base_pad = suggest_baseline_pad_pct(FONTS.mono, fudge=0.08)
style = Style(baseline_pad_pct=max(0.62, base_pad))

DUR = 1.5
W, H = 1920, 1080

# Build payloads (use examples from spec if present)
def example_of(type_id: str) -> str:
    t = next(x for x in SPEC["types"] if x["id"] == type_id)
    if "example" in t:
        return t["example"]
    if "examples" in t and t["examples"]:
        # turn list into bullets
        return " • ".join(t["examples"])
    return t.get("name", type_id)

clips = []

# Utility to wrap each text clip on a solid background so the segment is 1920x1080
def segment(clip):
    bg_color = tuple(np.array((12,16,24), dtype=np.uint8))  # 8-bit mỗi kênh
    bg = ColorClip((1920,1080), color=bg_color).with_duration(DUR)
    return CompositeVideoClip([bg, clip.with_duration(DUR)], size=(W, H))
other_args = {
    "debug_rect": True,
    "style": style
}
# Build one segment for each wrapper
clips.append(segment(title_clip(example_of("course_title"), SPEC, fonts=FONTS, duration=DUR, **other_args)))
clips.append(segment(subtitle_clip(example_of("subtitle"), SPEC, fonts=FONTS, duration=DUR, **other_args)))
clips.append(segment(lowerthird_clip(example_of("lowerthird_name"), SPEC, fonts=FONTS, duration=DUR, **other_args)))
clips.append(segment(section_marker_clip(example_of("section_marker"), SPEC, fonts=FONTS, duration=DUR)))
clips.append(segment(heading_h1_clip(example_of("heading_h1"), SPEC, fonts=FONTS, duration=DUR)))
clips.append(segment(heading_h2_clip(example_of("heading_h2"), SPEC, fonts=FONTS, duration=DUR)))
clips.append(segment(heading_h3_clip(example_of("heading_h3"), SPEC, fonts=FONTS, duration=DUR)))
clips.append(segment(body_bullet_clip(example_of("body_bullet"), SPEC, fonts=FONTS, duration=DUR)))
clips.append(segment(definition_term_clip(example_of("definition_term"), SPEC, fonts=FONTS, duration=DUR)))
clips.append(segment(step_label_clip(example_of("step_label"), SPEC, fonts=FONTS, duration=DUR)))
clips.append(segment(code_snippet_clip(example_of("code_snippet"), SPEC, fonts=FONTS, duration=DUR)))
clips.append(segment(equation_clip(example_of("equation"), SPEC, fonts=FONTS, duration=DUR)))
clips.append(segment(chart_labels_clip(example_of("chart_labels"), SPEC, fonts=FONTS, duration=DUR)))
clips.append(segment(data_labels_clip(example_of("data_labels"), SPEC, fonts=FONTS, duration=DUR)))
clips.append(segment(captions_clip(example_of("captions"), SPEC, fonts=FONTS, duration=DUR)))
clips.append(segment(speaker_label_clip(example_of("speaker_label"), SPEC, fonts=FONTS, duration=DUR)))
clips.append(segment(quiz_question_clip(example_of("quiz_question"), SPEC, fonts=FONTS, duration=DUR)))
clips.append(segment(quiz_choices_clip(example_of("quiz_choices"), SPEC, fonts=FONTS, duration=DUR)))
clips.append(segment(quiz_feedback_clip(example_of("quiz_feedback"), SPEC, fonts=FONTS, duration=DUR)))
clips.append(segment(cta_clip(example_of("cta"), SPEC, fonts=FONTS, duration=DUR)))
clips.append(segment(progress_ui_clip(example_of("progress_ui"), SPEC, fonts=FONTS, duration=DUR)))
clips.append(segment(timestamp_clip(example_of("timestamp"), SPEC, fonts=FONTS, duration=DUR)))
clips.append(segment(legal_credits_clip(example_of("legal_credits"), SPEC, fonts=FONTS, duration=DUR)))

final = concatenate_videoclips(clips, method="compose")
print("Done -> all_text_types_demo.mp4")

Done -> all_text_types_demo.mp4


In [90]:
final.preview()

In [80]:
# 1) Tạo text clip gốc
txt = TextClip(text="y g p q ạ ậ ỵ", font=FONTS.sans, font_size=64,
               color="white", method="label", transparent=True)

# 2) Lưu frame TRƯỚC khi pad
txt.save_frame("t0_raw.png", t=0)     # nếu ảnh này đã mất đuôi → cắt ở render TextClip

# 3) Pad trong suốt
pad = ColorClip((txt.w, txt.h + 80), color=(0,0,0)).with_opacity(0.0)   # bg alpha 0
padded = CompositeVideoClip([pad, txt.with_position((0, 0))], size=(txt.w, txt.h+80))
padded.save_frame("t1_padded.png", t=0) # nếu t0 OK mà t1 vẫn “mất” → do mask pad hợp sai

# 4) Đặt lên nền (composite)
bg = ColorClip((1920,1080), color=(15,18,24)).with_duration(1)
CompositeVideoClip([bg, padded.with_position((200, 200))], size=(1920,1080)).save_frame("t2_on_bg.png", t=0)


## Multi Line

In [91]:
# ========= MULTI-LINE: WRAP MODE =========
def wrapped_text_clip(
    element_id: str,
    text: str,
    spec: Dict[str, Any],
    variant: str = "primary",
    fonts: Optional[Fonts] = None,
    style: Optional[Style] = None,
    duration: Optional[float] = None,
    max_lines: Optional[int] = None,
    overflow: str = "shrink",        # "none" | "shrink"
    min_font_size: int = 18,
    debug_rect: bool = False,
) -> VideoClip:
    """
    Multi-line dạng 'wrap': tự bọc theo width của rect.
    - max_lines: giới hạn số dòng (ước lượng theo chiều cao).
    - overflow='shrink': nếu vẫn tràn -> co tỉ lệ, không nhỏ hơn min_font_size.
    """
    fonts = fonts or Fonts()
    style = style or Style()

    # lấy layout/rect/align/px từ spec
    t = next(x for x in spec["types"] if x["id"] == element_id)
    layout = t["layout"]["alternate" if variant == "alternate" else "primary"]
    rect = tuple(layout["rect"])
    h_align = layout.get("align", "left")
    px = int(t["size"]["common"])

    # tạo text clip (kế thừa cơ chế pad top/bottom + interline đã có)
    base = _mk_text_clip(text, rect, (fonts.mono if t["size"].get("mono") else fonts.sans), px, h_align, style)
    pad_top = int(max(2, ceil(px * style.top_pad_pct)))
    pad_bottom = int(max(4, ceil(px * style.baseline_pad_pct)))
    clip = _pad_transparent(base, 0, pad_top, 0, pad_bottom)

    # giới hạn theo số dòng (ước lượng bằng chiều cao)
    allowed_h = rect[3]
    if max_lines:
        line_h = px + style.interline
        allowed_h = min(allowed_h, int(style.top_pad_pct*px + max_lines*line_h + style.baseline_pad_pct*px))

    # co nếu tràn
    if overflow == "shrink" and clip.h > allowed_h:
        scale = allowed_h / float(clip.h)
        est_font = px * scale
        if est_font < min_font_size:
            scale = min_font_size / float(px)
        clip = clip.resized(new_size=(max(1, int(clip.w*scale)), max(1, int(clip.h*scale))))

    # vertical-align giống logic phần tử thường
    if element_id in {"section_marker", "equation", "quiz_question", "quiz_feedback", "cta"}:
        v_align = "center"
    elif element_id in {"captions"}:
        v_align = "bottom"
    else:
        v_align = "top"

    if element_id == "captions":
        clip = _caption_bg(clip, pad=max(style.pad, 14), opacity=0.65)

    if style.opacity != 1.0:
        clip = clip.with_opacity(style.opacity)
    if duration is not None:
        clip = clip.with_duration(duration)

    placed = _place_in_rect(clip, rect, h_align=h_align, v_align=v_align)

    if debug_rect:
        x, y, w, h = rect
        box = ColorClip((w, h), color=(255, 255, 255)).with_opacity(0.15).with_duration(placed.duration or duration)
        box = box.with_position((x, y))
        return CompositeVideoClip([box, placed], size=(1920, 1080))

    return placed


# ========= MULTI-LINE: STRUCTURED (STAGGERED) =========
def structured_multiline_clip(
    element_id: str,
    lines: list[str],                 # danh sách dòng/mục
    spec: Dict[str, Any],
    variant: str = "primary",
    fonts: Optional[Fonts] = None,
    style: Optional[Style] = None,
    duration: float = 3.0,            # tổng thời lượng segment
    stagger: float = 0.25,            # độ trễ giữa các dòng
    gap_px: Optional[int] = None,     # khoảng cách giữa các dòng (mặc định theo px)
    h_align_override: Optional[str] = None,
    v_align: str = "top",             # "top" | "center" | "bottom" (bố trí khối dòng trong rect)
    debug_rect: bool = False,
) -> VideoClip:
    """
    Multi-line 'thật sự': mỗi dòng là một TextClip riêng, xuất hiện so-le theo thời gian.
    - Không phá layout tổng thể: mọi dòng vẫn nằm trong rect của element_id.
    - Nếu tổng chiều cao > rect.h, sẽ co tỉ lệ tất cả dòng cho vừa (uniform scale).
    """
    fonts = fonts or Fonts()
    style = style or Style()

    t = next(x for x in spec["types"] if x["id"] == element_id)
    layout = t["layout"]["alternate" if variant == "alternate" else "primary"]
    rect = tuple(layout["rect"])
    base_align = layout.get("align", "left")
    h_align = h_align_override or base_align
    px = int(t["size"]["common"])

    # build từng dòng (pad top/bottom để tránh cắt)
    line_clips = []
    for s in lines:
        base = _mk_text_clip(s, rect, (fonts.mono if t["size"].get("mono") else fonts.sans), px, h_align, style)
        pad_top = int(max(2, ceil(px * style.top_pad_pct)))
        pad_bottom = int(max(4, ceil(px * style.baseline_pad_pct)))
        c = _pad_transparent(base, 0, pad_top, 0, pad_bottom)
        line_clips.append(c)

    # đo tổng chiều cao + khoảng cách
    gap = gap_px if gap_px is not None else max(6, int(px * 0.25))
    total_h = sum(c.h for c in line_clips) + gap * (len(line_clips) - 1)

    # co tỉ lệ nếu vượt rect.h
    rect_h = rect[3]
    scale = 1.0
    if total_h > rect_h:
        scale = rect_h / float(total_h)
        for i in range(len(line_clips)):
            c = line_clips[i]
            line_clips[i] = c.resized(new_size=(max(1, int(c.w*scale)), max(1, int(c.h*scale))))
        gap = int(gap * scale)
        total_h = sum(c.h for c in line_clips) + gap * (len(line_clips) - 1)

    # tính toạ độ Y bắt đầu theo v_align
    x, y, w, h = rect
    if v_align == "center":
        y0 = y + max(0, (h - total_h) // 2)
    elif v_align == "bottom":
        y0 = y + max(0, h - total_h)
    else:
        y0 = y

    # đặt từng dòng + gán thời gian xuất hiện (with_start)
    placed = []
    y_cursor = y0
    for i, c in enumerate(line_clips):
        # đặt ngang theo h_align bên trong rect (chiều cao rect tạm = c.h)
        placed_i = _place_in_rect(c, (x, y_cursor, w, c.h), h_align=h_align, v_align="top")
        start_i = i * max(0.0, float(stagger))
        live_dur = max(0.001, duration - start_i)  # giữ tới cuối segment
        placed.append(placed_i.with_start(start_i).with_duration(live_dur))
        y_cursor += c.h + gap

    comp = CompositeVideoClip(placed, size=(1920, 1080)).with_duration(duration)

    if debug_rect:
        box = ColorClip((w, h), color=(255, 255, 255)).with_opacity(0.12).with_duration(duration).with_position((x, y))
        comp = CompositeVideoClip([box, comp], size=(1920, 1080)).with_duration(duration)
    return comp


## Wrapper của Multi line

In [92]:
# ========= WRAPPERS CHO 2 CHẾ ĐỘ =========
# Wrap: dùng cho các loại văn bản “đơn”
def title_wrap(text, spec, **kw):           return wrapped_text_clip("course_title", text, spec, **kw)
def subtitle_wrap(text, spec, **kw):        return wrapped_text_clip("subtitle", text, spec, **kw)
def definition_wrap(text, spec, **kw):      return wrapped_text_clip("definition_term", text, spec, **kw)
def eq_wrap(text, spec, **kw):              return wrapped_text_clip("equation", text, spec, **kw)
def caption_wrap(text, spec, **kw):         return wrapped_text_clip("captions", text, spec, **kw)
def h1_wrap(text, spec, **kw):              return wrapped_text_clip("heading_h1", text, spec, **kw)
def h2_wrap(text, spec, **kw):              return wrapped_text_clip("heading_h2", text, spec, **kw)
def h3_wrap(text, spec, **kw):              return wrapped_text_clip("heading_h3", text, spec, **kw)

# Structured: dùng cho Bullets/Quiz choices… hiển thị so-le
def bullets_staggered_clip(items, spec, **kw):      return structured_multiline_clip("body_bullet", items, spec, **kw)
def quiz_choices_staggered_clip(items, spec, **kw): return structured_multiline_clip("quiz_choices", items, spec, **kw)


## Test Multi line wrapper

In [93]:
from moviepy import ColorClip, CompositeVideoClip, concatenate_videoclips

# 1) Wrap (title dài) – giới hạn 2 dòng, nếu tràn co chữ
seg_title = title_wrap(
  "Python cho E-Learning: Tư duy hệ thống & MoviePy v2 — thực chiến",
  SPEC,
  fonts=FONTS,
  style=Style(color="white", interline=8),
  max_lines=2, overflow="shrink",
  duration=3
)

# 2) Structured bullets – 4 dòng, stagger 0.25s/dòng, căn trái, giữ layout
bullets = [
  "Module hoá pipeline render",
  "Idempotent + retry an toàn",
  "Chuẩn hoá template & style",
  "Render phân tán với hàng đợi"
]
seg_bullets = bullets_staggered_clip(
  bullets, SPEC,
  fonts=FONTS,
  style=Style(color="#cfe2ff", interline=8),
  duration=3, stagger=0.25, v_align="top",h_align_override="left"
)

# 3) Structured quiz choices – căn trái, mỗi lựa chọn xuất hiện cách nhau 0.2s
choices = ["A) Async render", "B) fontsize → font_size", "C) .with_duration mới", "D) Tất cả đáp án"]
seg_quiz = quiz_choices_staggered_clip(
  choices, SPEC,
  fonts=FONTS, style=Style(color="white", interline=8),
  duration=3, stagger=0.2, v_align="top",h_align_override="left"
)

bg = ColorClip((1920,1080), color=(12,16,24))
clips = []
for c in [seg_title, seg_bullets, seg_quiz]:
    clips.append(CompositeVideoClip([bg.with_duration(3), c], size=(1920,1080)).with_duration(3))

final = concatenate_videoclips(clips, method="compose")
final.preview()

## Trash Section

In [59]:
FONTS

Fonts(sans='D:/ThienPV/code/demo/assets/Inter/Inter-Italic-VariableFont_opsz,wght.ttf', mono='D:/ThienPV/code/demo/assets/font/JetBrains_Mono/static/JetBrainsMono-Regular.ttf')

In [60]:
from fontTools.ttLib import TTFont
font = TTFont(FONTS.sans)
hhea = font["hhea"]
print(hhea.ascent, hhea.descent)  # descent thường là số âm


1984 -494


MoviePy - Building video test.mp4.
MoviePy - Writing video test.mp4



MoviePy - Done !
MoviePy - video ready test.mp4
